In [18]:
import random
import matplotlib.pyplot as plt
import os
from flask import Flask, render_template

# Set the template folder explicitly
template_dir = os.path.abspath('templates')
app = Flask(__name__, template_folder=template_dir)
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/genetic_algorithm')
def genetic_algorithm():
    # Dataset
    dataset = [
        [199.0, 32.0],
        [75.0, 152.0],
        [121.0, 40.0],
        [38.0, 199.0],
        [162.0, 114.0],
        [199.0, 106.0],
        [125.0, 147.0],
        [118.0, 93.0],
        [26.0, 142.0],
        [21.0, 119.0],
        [50.0, 139.0],
        [84.0, 19.0],
        [136.0, 18.0],
        [35.0, 124.0],
        [32.0, 134.0],
        [126.0, 2.0],
        [153.0, 166.0],
        [23.0, 144.0],
        [84.0, 69.0],
        [199.0, 77.0],
        [97.0, 10.0],
        [14.0, 121.0],
        [156.0, 1.0],
        [62.0, 132.0],
        [183.0, 1.0]

    ]

    # Genetic Algorithm Parameters
    population_size = 50  # Number of routes in each generation
    mutation_rate = 0.01  # Probability of mutation
    num_generations = 1000  # Number of generations

    # Calculate the Euclidean distance between two cities
    def calculate_distance(city_a, city_b):
        x1, y1 = city_a
        x2, y2 = city_b
        return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

    # Generate an initial population randomly
    def generate_population():
        population = []
        for _ in range(population_size):
            route = list(range(len(dataset)))
            random.shuffle(route)
            population.append(route)
        return population

    # Calculate the total distance of a route
    def calculate_route_distance(route):
        total_distance = 0
        for i in range(len(route) - 1):
            city_a = dataset[route[i]]
            city_b = dataset[route[i + 1]]
            total_distance += calculate_distance(city_a, city_b)
        return total_distance

    # Perform mutation on a route
    def mutate(route):
        for _ in range(len(route)):
            if random.random() < mutation_rate:
                i, j = random.sample(range(len(route)), 2)
                route[i], route[j] = route[j], route[i]

    # Evolve the population through generations
    def evolve(population):
        for _ in range(num_generations):
            # Calculate fitness of each route
            fitness = [1 / calculate_route_distance(route) for route in population]
            total_fitness = sum(fitness)

            # Select parents for crossover
            parents = random.choices(population, weights=fitness, k=population_size)

            # Create new generation through crossover
            new_generation = []
            for _ in range(population_size):
                parent_a, parent_b = random.sample(parents, 2)
                child = parent_a.copy()
                for i in range(len(child)):
                    if child[i] not in parent_b:
                        j = child.index(parent_b[i])
                        child[i], child[j] = child[j], child[i]
                new_generation.append(child)

            # Apply mutation to the new generation
            for route in new_generation:
                mutate(route)

            # Update the population
            population = new_generation

        return population

    # Main program
    population = generate_population()
    final_population = evolve(population)

    # Find the best route in the final population
    best_route = min(final_population, key=calculate_route_distance)
    best_distance = calculate_route_distance(best_route)
    
    # Append the index of the first city to the end of the route
    best_route.append(best_route[0])

    # Plot the best route
    x = [point[0] for point in dataset]
    y = [point[1] for point in dataset]

    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, color='blue')
    plt.plot([dataset[city][0] for city in best_route], [dataset[city][1] for city in best_route], color='red',
             linewidth=2)
    plt.title('TSP Solution')
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')

    # Create the static directory if it doesn't exist
    os.makedirs('static', exist_ok=True)

    # Save the plot image to the static directory
    plot_path = 'static/plot.png'
    plt.savefig(plot_path)

    return render_template('genetic_algorithm.html', plot_path=plot_path, best_distance=best_distance)


if __name__ == '__main__':
    app.run(port=5678)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5678
Press CTRL+C to quit
127.0.0.1 - - [21/Jun/2023 01:15:32] "GET /genetic_algorithm HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2023 01:15:32] "GET /static/plot.png HTTP/1.1" 200 -
